In [1]:
# 标定流程如下图所示，
# 首先要拍摄标定板的图像，拍摄数量一般在 10 张左右，
# 要保证标定板完全出现在图片中，
# 同时尽量选取多个角度拍摄，
# 拍摄完成后，要寻找标定板的圆点位置
# 如果是棋盘格，就是找棋盘格的角点也就是格子的 4 个角，
# 然后将圆点在图像上的坐标和现实中的坐标相匹配，
# 最后计算得到相机参数，完成标定。

In [2]:
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt
import cvzone
import os

In [3]:
#求输入数据的归一化矩阵
def normalizing_input_data(coor_data):
    x_avg =np.mean(coor_data[:,0])
    y_avg=np.mean(coor_data[:,1])
    sx =np.sqrt(2)/ np.std(coor_data[:,0])
    sy =np.sqrt(2)/ np.std(coor_data[:,1])
    norm_matrix = np.matrix([[sx,0,-sx * x_avg],
                             [0, sy, -sy * y_avg],
                             [0,0,1]])
    return norm_matrix

In [4]:
#求单应矩阵H
def get_H(pic_coor,real_coor):
    #获得归一化矩阵
    pic_norm_mat =normalizing_input_data(pic_coor)
    real_norm_mat =normalizing_input_data(real_coor)
    M=[]
    for i in range(len(pic_coor)):
        # 转换为齐次坐标
        single_pic_coor = np.array([pic_coor[i][0],pic_coor[i][1],1])
        single_real_coor = np.array([real_coor[i][0],real_coor[i][1],1])
        # 坐标归一化
        pic_norm = np.dot(pic_norm_mat, single_pic_coor)
        real_norm=np.dot(real_norm_mat,single_real_coor)
        # 构造M矩阵
        M.append(np.array([-real_norm.item(0),-real_norm.item(1),-1,0,0,0,pic_norm.item(0) * real_norm.item(0),pic_norm.item(0) * real_norm.item(1),pic_norm.item(0)]))
        M.append(np.array([0,0,0,
        -real_norm.item(0), -real_norm.item(1),-1,
        pic_norm.item(1) * real_norm.item(0),pic_norm.item(1) * real_norm.item(1) ,pic_norm.item(1)]))
    #·利用SVD求解N*h =0中h的解
    U,S,VT = np.linalg.svd((np .array(M, dtype='float')).reshape((-1,9)))
    #最小的奇异值对应的奇异向量,S求出来按大小排列的，最后的最小
    H=VT[-1].reshape((3,3))
    H = np.dot(np.dot(np.linalg.inv(pic_norm_mat),H),real_norm_mat)
    H/=H[-1,-1]
    return H

In [5]:
# 返回H,pic_coor为角点在图像上的坐标,real_coor为角点在现实中的坐标
def get_Homography(pic_coor,real_coor):
    refined_homographies = []
    for i in range(len(pic_coor)):
        H = get_H(pic_coor[i],real_coor[i]).reshape(-1,1)
        refined_homographies.append(H)
    return np.array(refined_homographies)

In [6]:
# 返回pq位置对应的v向量
def create_v(p,q,H):
    H = H.reshape(3,3)
    return np.array([
        H[0][p]*H[0][q],
        H[0][p]*H[1][q]+H[1][p]*H[0][q],
        H[1][p]*H[1][q],
        H[2][p]*H[0][q]+H[0][p]*H[2][q],
        H[2][p]*H[1][q]+H[1][p]*H[2][q],
        H[2][p]*H[2][q]
    ])
    

In [7]:
# 返回相机内参矩阵A
def get_intrinsic_param(H):
    # 构建V矩阵
    V = np.array([])
    for i in range(len(H)):
        V = np.append(V,np.array([create_v(0,1,H[i]),create_v(0,0,H[i])-create_v(1,1,H[i])]))
    #利用SVD求解Vb=0中b的解
    U,S,VT = np.linalg.svd((np.array(V,dtype='float')).reshape((-1,6)))
    #最小的奇异值对应的奇异向量,S求出来按大小排列的，最后的最小
    b = VT[-1]
    #求出相机内参矩阵A
    w = b[0]*b[2]*b[5]-b[1]*b[1]*b[5]-b[0]*b[4]*b[4]+2*b[1]*b[3]*b[4]-b[2]*b[3]*b[3]
    d = b[0]*b[2]-b[1]*b[1]
    print(b[0])
    print(d)
    alpha = np.sqrt(w/(d*b[0]))
    beta = np.sqrt(w/(d*d)*b[0])
    gamma = np.sqrt(w/(d*d)*b[0])
    uc = (b[1]*b[4]-b[2]*b[3])/d
    vc = (b[1]*b[3]-b[0]*b[4])/d
    A = np.array([
        [alpha,gamma,uc],
        [0,beta,vc],
        [0,0,1]
    ])
    return A

    

In [8]:
# 返回每一幅图的外参矩阵[R|t]
def get_extrinsic_param(H,intrinsics_param):
    extrinsic_param = []
    inv_intrinsics_param = np.linalg.inv(intrinsics_param)
    for i in range(len(H)):
        h0 = (H[i].reshape(3,3))[:,0]
        h1 = (H[i].reshape(3,3))[:,1]
        h2 = (H[i].reshape(3,3))[:,2]
        scale_factor = 1/np.linalg.norm(np.dot(inv_intrinsics_param,h0))
        r0 = scale_factor * np.dot(inv_intrinsics_param,h0)
        r1 = scale_factor * np.dot(inv_intrinsics_param,h1)
        r2 = np.cross(r0,r1)
        t = scale_factor * np.dot(inv_intrinsics_param,h2)

        R = np.array([r0,r1,r2,t]).transpose()
        extrinsic_param.append(R)
    return np.array(extrinsic_param)



In [9]:
file_dir = "./img"
pic_name = os.listdir(file_dir)

In [10]:
# 角点为7*7
w = 7
h = 7


In [11]:
cross_corners = [w,h]
real_coor = np.zeros((cross_corners[0]*cross_corners[1],2),np.float32)
real_coor[:,:2] = np.mgrid[0:w,0:h].T.reshape(-1,2)
real_points = []
real_points_x_y = []
pic_points = []


In [12]:
for pic in pic_name:
    # 读取照片
    pic_path = os.path.join(file_dir,pic)
    pic_data = cv2.imread(pic_path)
    # 寻找棋盘角点，ret为是否找到角点，pic_coor为角点在图像上的坐标，54*1*2
    ret,pic_coor = cv2.findCirclesGrid(pic_data,(cross_corners[0],cross_corners[1]),None)
    if ret:
        # 添加对应3D-2D坐标
        pic_coor = pic_coor.reshape(-1,2)
        pic_points.append(pic_coor)
        real_points.append(real_coor)
        real_points_x_y.append(real_coor[:,:2])


In [13]:
# 求单应矩阵H
H = get_Homography(pic_points,real_points)




In [14]:
# 求相机内参矩阵A
A = get_intrinsic_param(H)

1.974588332872603e-07
3.880150344691412e-14


In [15]:
# 求外参矩阵[R|t]
R_t = get_extrinsic_param(H,A)

In [16]:
# 打印相机参数
print("相机内参矩阵A为：\n",A)
print("外参矩阵[R|t]为：\n")
for item in R_t:
    print(item)

相机内参矩阵A为：
 [[2.16098454e+03 2.16622692e+03 4.05752470e+02]
 [0.00000000e+00 2.16622692e+03 4.79795572e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
外参矩阵[R|t]为：

[[ 9.76117180e-01 -8.55811295e-01 -1.91061059e-01 -8.32276781e-01]
 [-7.98332943e-02  9.25113970e-01 -2.23617673e-01 -4.71864364e+00]
 [ 2.02044292e-01  5.19465150e-02  8.34697404e-01  6.31824885e+01]]
[[ 0.97463237 -0.83512084 -0.1937702  -0.34298995]
 [-0.09076814  0.9051215  -0.26319898 -5.45171027]
 [ 0.2045798   0.0947538   0.80635835 61.4269825 ]]
[[ 0.98497428 -0.86927704 -0.15115062  3.1552853 ]
 [-0.06993941  0.92718957 -0.20405066 -5.19861749]
 [ 0.15790548  0.06780588  0.85246116 63.78989974]]
[[ 0.9933829  -0.819453   -0.0710197   5.9303125 ]
 [-0.09161532  0.89547781 -0.15432331 -7.82774659]
 [ 0.06926079  0.09821726  0.81447789 60.92334428]]
[[ 0.97893318 -0.8432197  -0.17518641 -0.3338121 ]
 [-0.08655968  0.90932308 -0.23543552 -3.56225855]
 [ 0.18492497  0.08121406  0.81717771 62.95051423]]
[[ 9.96421972